# Welcome to Newton Book

### Powered By Edison

Newton Book is interactive web-based tool for data analysis, research, and teaching. Write code, create visualizations, and blend it with text to tell data-driven stories. Enjoy exploring and creating with Newton book!

In [1]:
print("Hello, Begin Your Data Journey")

Hello, Begin Your Data Journey


In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn 
import torch.optim as optim


In [27]:
#generate data 

np.random.seed(0)
torch.manual_seed(0)

def generate_stock_data(n_days=500):
    prices = [100]
    for _ in range(n_days - 1):
        drift = np.random.normal(0.05, 0.5)
        prices.append(prices[-1] + drift)

    df = pd.DataFrame({"price": prices})
    df["return"] = df["price"].diff().fillna(0)
    df["direction"] = (df["return"] > 0).astype(int)
    return df

df = generate_stock_data()

WINDOW = 5

X, y = [], []

for i in range(len(df) - WINDOW):
    X.append(df["return"].iloc[i:i+WINDOW].values)
    y.append(df["direction"].iloc[i+WINDOW])

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)




(495, 5) (495,)


In [28]:

X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [37]:
import torch.nn as nn

class StockRNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        #create a neural network architecture with first layer as rnn 1 recurrent layer containing 32 neurons 
        #and a linear layer which will take input from the hidden layer and give binary label as output 
        self.rnn = nn.RNN(input_size=1, hidden_size=32, batch_first=True)
        
        self.linear = nn.Linear(32, 1)
        
        self.acti = nn.Tanh()

    def forward(self, x):
        # call the rnn layer
        _, h_n = self.rnn(x)
        
        x = self.acti(self.linear(h_n.squeeze(0)).squeeze(-1))
        return x



In [39]:
model = StockRNN()
criterion = nn.CrossEntropyLoss()

# add learning rate 0.01 and model parameter in the adam function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(600):
    optimizer.zero_grad()
    logits = model(X_tensor)
    loss = criterion(logits, y_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 1576.1786
Epoch 50, Loss: 1566.3040
Epoch 100, Loss: 1529.6169
Epoch 150, Loss: 1492.0676
Epoch 200, Loss: 1472.0272
Epoch 250, Loss: 1462.9136
Epoch 300, Loss: 1457.8938
Epoch 350, Loss: 1455.4243
Epoch 400, Loss: 1454.2350
Epoch 450, Loss: 1452.9780
Epoch 500, Loss: 1451.9099
Epoch 550, Loss: 1451.6423


In [40]:
with torch.no_grad():
    probs = torch.sigmoid(model(X_tensor))
    preds = (probs > 0.5).float()
    accuracy = (preds == y_tensor).float().mean()

print("Training accuracy:", accuracy.item())


Training accuracy: 0.965656578540802


In [41]:
round(accuracy.item(),2)

0.97

In [42]:
single_return_seq = [1000 , 700 , 300 , 200  , 500  ]
x_single = torch.tensor(
    single_return_seq,
    dtype=torch.float32
).unsqueeze(0).unsqueeze(-1)
model.eval()

with torch.no_grad():
    logit = model(x_single)
    prob = torch.sigmoid(logit)

print("Probability of price going UP:", prob.item())


Probability of price going UP: 0.2689424455165863
